In [1]:
import gym
import keras

print(gym.__version__)
print(keras.__version__)
print(keras.backend.backend())

Using Theano backend.


0.15.3
2.2.4
theano


In [2]:
import math
import random
import numpy as np
from collections import deque

In [5]:
#using gym to run an environment 

env = gym.make('CartPole-v0')
for i_epsd in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episodes finished after {} time steps".format(t+1))
env.reset()

[-0.02690064  0.0238192  -0.02826297  0.03227096]
[-0.02642425 -0.17088628 -0.02761755  0.31590435]
[-0.02984198 -0.36560419 -0.02129946  0.59975129]
[-0.03715406 -0.56042177 -0.00930444  0.88564982]
[-0.0483625  -0.75541618  0.00840856  1.1753933 ]
[-0.06347082 -0.56040449  0.03191642  0.88535819]
[-0.07467891 -0.36573005  0.04962359  0.60287709]
[-0.08199351 -0.17133602  0.06168113  0.32622834]
[-0.08542023 -0.36727945  0.0682057   0.6377073 ]
[-0.09276582 -0.17317156  0.08095984  0.36725939]
[-0.09622925 -0.36934501  0.08830503  0.68433192]
[-0.10361615 -0.56557485  0.10199167  1.00345776]
[-0.11492765 -0.76190045  0.12206082  1.3263494 ]
[-0.13016566 -0.95833354  0.14858781  1.65460364]
[-0.14933233 -1.15484469  0.18167988  1.98964661]
Episodes finished after 15 time steps
[-0.17242923 -0.96203409  0.22147282  1.75831128]
Episodes finished after 16 time steps
[-0.19166991 -0.77000098  0.25663904  1.54187925]
Episodes finished after 17 time steps
[-0.20706993 -0.57871319  0.28747663

[-0.0803066  -0.38997542  0.10469432  0.57109119]
[-0.08810611 -0.19646537  0.11611615  0.31313937]
[-0.09203541 -0.39303353  0.12237893  0.64006759]
[-0.09989608 -0.58962995  0.13518028  0.96864637]
[-0.11168868 -0.39655637  0.15455321  0.72130018]
[-0.11961981 -0.59344019  0.16897922  1.05836371]
[-0.13148861 -0.40091044  0.19014649  0.8231262 ]
[-0.13950682 -0.20882734  0.20660901  0.59576136]
Episodes finished after 18 time steps
[-0.14368337 -0.40615012  0.21852424  0.9457624 ]
Episodes finished after 19 time steps
[-0.15180637 -0.21453306  0.23743949  0.72890827]
Episodes finished after 20 time steps
[-0.15609703 -0.02334186  0.25201765  0.51932084]
Episodes finished after 21 time steps
[-0.15656387 -0.22096228  0.26240407  0.87969897]
Episodes finished after 22 time steps
[-0.16098312 -0.03017671  0.27999805  0.67958126]
Episodes finished after 23 time steps
[-0.16158665  0.16019695  0.29358968  0.48638965]
Episodes finished after 24 time steps
[-0.15838271 -0.03761873  0.303317

[-0.10442166 -0.41604938  0.15201379  0.83670216]
[-0.11274265 -0.22329409  0.16874783  0.5954236 ]
[-0.11720853 -0.03088557  0.1806563   0.36028706]
[-0.11782624  0.16127036  0.18786204  0.12956841]
[-0.11460084 -0.03597641  0.19045341  0.47514009]
[-0.11532036  0.15601799  0.19995621  0.24801123]
[-0.1122     -0.04131423  0.20491644  0.59650803]
Episodes finished after 23 time steps
[-0.11302629  0.15044018  0.2168466   0.37471895]
Episodes finished after 24 time steps
[-0.11001749 -0.04699688  0.22434098  0.72719325]
Episodes finished after 25 time steps
[-0.11095742  0.14443611  0.23888484  0.51264385]
Episodes finished after 26 time steps
[-0.1080687   0.33553873  0.24913772  0.30369625]
Episodes finished after 27 time steps
[-0.10135793  0.52640682  0.25521165  0.09872469]
Episodes finished after 28 time steps
[-0.09082979  0.7171385   0.25718614 -0.10388574]
Episodes finished after 29 time steps
[-0.07648702  0.90783245  0.25510842 -0.30573674]
Episodes finished after 30 time st

[-0.56916229 -2.33987816  1.1141076   5.32048579]
Episodes finished after 25 time steps
[-0.61595985 -2.1375699   1.22051731  5.45053555]
Episodes finished after 26 time steps
[-0.65871125 -1.93304942  1.32952803  5.62140815]
Episodes finished after 27 time steps
[-0.69737224 -2.09068634  1.44195619  5.96338997]
Episodes finished after 28 time steps
[-0.73918596 -2.24231682  1.56122399  6.2841763 ]
Episodes finished after 29 time steps
[-0.7840323  -2.02472604  1.68690751  6.5750386 ]
Episodes finished after 30 time steps
[-0.82452682 -2.16609943  1.81840829  6.84249174]
Episodes finished after 31 time steps
[-0.86784881 -1.93891018  1.95525812  7.21104741]
Episodes finished after 32 time steps
[-0.90662701 -2.07355229  2.09947907  7.40783718]
Episodes finished after 33 time steps
[-0.94809806 -1.83876475  2.24763581  7.83933699]
Episodes finished after 34 time steps
[-0.98487335 -1.97412599  2.40442255  7.94135481]
Episodes finished after 35 time steps
[-1.02435587 -1.7383052   2.5632

[-0.2368987   0.07480245  0.65236828  2.02952834]
Episodes finished after 32 time steps
[-0.23540265  0.26043962  0.69295885  1.98673254]
Episodes finished after 33 time steps
[-0.23019386  0.06651957  0.7326935   2.39833584]
Episodes finished after 34 time steps
[-0.22886346 -0.12568767  0.78066022  2.80930812]
Episodes finished after 35 time steps
[-0.23137722  0.06092012  0.83684638  2.81734722]
Episodes finished after 36 time steps
[-0.23015881 -0.12796435  0.89319333  3.22542681]
Episodes finished after 37 time steps
[-0.2327181  -0.31392483  0.95770186  3.62935122]
Episodes finished after 38 time steps
[-0.2389966  -0.12432174  1.03028889  3.70615867]
Episodes finished after 39 time steps
[-0.24148303  0.06573828  1.10441206  3.81154918]
Episodes finished after 40 time steps
[-0.24016827  0.25661725  1.18064304  3.94540411]
Episodes finished after 41 time steps
[-0.23503592  0.08145857  1.25955112  4.31723743]
Episodes finished after 42 time steps
[-0.23340675 -0.08921688  1.3458

...

[ 0.6286435   2.31457173 -1.10603116 -5.3855554 ]
Episodes finished after 55 time steps
[ 0.67493493  2.48044578 -1.21374227 -5.75989022]
Episodes finished after 56 time steps
[ 0.72454385  2.273018   -1.32894007 -5.92659893]
Episodes finished after 57 time steps
[ 0.77000421  2.06248182 -1.44747205 -6.13640529]
Episodes finished after 58 time steps
[ 0.81125385  2.21211353 -1.57020016 -6.45578214]
Episodes finished after 59 time steps
[ 0.85549612  1.99241502 -1.6993158  -6.74958562]
Episodes finished after 60 time steps
[ 0.89534442  2.13161666 -1.83430751 -7.01439956]
Episodes finished after 61 time steps
[ 0.93797675  2.26751787 -1.9745955  -7.24515336]
Episodes finished after 62 time steps
[ 0.98332711  2.03453652 -2.11949857 -7.65282111]
Episodes finished after 63 time steps
[ 1.02401784  2.16744724 -2.27255499 -7.79967709]
Episodes finished after 64 time steps
[ 1.06736678  2.30432986 -2.42854854 -7.89165797]
Episodes finished after 65 time steps
[ 1.11345338  2.06971286 -2.5863

[ 1.07517941  1.5495047  -3.86440999 -7.91629017]
Episodes finished after 67 time steps
[ 1.1061695   1.40651911 -4.02273579 -7.88265614]
Episodes finished after 68 time steps
[ 1.13429988  1.64505957 -4.18038891 -7.42818083]
Episodes finished after 69 time steps
[ 1.16720107  1.5099419  -4.32895253 -7.2776226 ]
Episodes finished after 70 time steps
[ 1.19739991  1.37613583 -4.47450498 -7.08005876]
Episodes finished after 71 time steps
[ 1.22492263  1.60617226 -4.61610616 -6.71302715]
Episodes finished after 72 time steps
[ 1.25704607  1.4663229  -4.7503667  -6.44055529]
Episodes finished after 73 time steps
[ 1.28637253  1.32166605 -4.87917781 -6.13852866]
Episodes finished after 74 time steps
[ 1.31280585  1.5354788  -5.00194838 -5.90185321]
Episodes finished after 75 time steps
[ 1.34351543  1.37948373 -5.11998544 -5.55328058]
Episodes finished after 76 time steps
[ 1.3711051   1.58437225 -5.23105105 -5.40519407]
Episodes finished after 77 time steps
[ 1.40279255  1.41706452 -5.3391

[-0.8722704  -2.07145109  5.22596931  3.63593125]
Episodes finished after 68 time steps
[-0.91369942 -2.26114025  5.29868794  3.51965097]
Episodes finished after 69 time steps
[-0.95892223 -2.07873692  5.36908096  3.12337002]
Episodes finished after 70 time steps
[-1.00049696 -2.26587301  5.43154836  3.06188721]
Episodes finished after 71 time steps
[-1.04581442 -2.45301738  5.4927861   3.02561616]
Episodes finished after 72 time steps
[-1.09487477 -2.26405422  5.55329843  2.61726885]
Episodes finished after 73 time steps
[-1.14015586 -2.07249365  5.6056438   2.20709324]
Episodes finished after 74 time steps
[-1.18160573 -1.87891269  5.64978567  1.79655757]
Episodes finished after 75 time steps
[-1.21918398 -1.68381377  5.68571682  1.38666195]
Episodes finished after 76 time steps
[-1.25286026 -1.48762001  5.71345006  0.97796344]
Episodes finished after 77 time steps
[-1.28261266 -1.67278906  5.73300933  1.05325783]
Episodes finished after 78 time steps
[-1.31606844 -1.47578174  5.7540

[ 0.65119267  0.27158496 -6.19984091 -5.55421346]
Episodes finished after 81 time steps
[ 0.65662436  0.07787698 -6.31092518 -5.24018518]
Episodes finished after 82 time steps
[ 0.6581819  -0.11757939 -6.41572889 -4.9552679 ]
Episodes finished after 83 time steps
[ 0.65583032 -0.31373788 -6.51483424 -4.70246476]
Episodes finished after 84 time steps
[ 0.64955556 -0.12110722 -6.60888354 -5.05119008]
Episodes finished after 85 time steps
[ 0.64713341  0.06897361 -6.70990734 -5.41539308]
Episodes finished after 86 time steps
[ 0.64851289 -0.13009154 -6.8182152  -5.26525478]
Episodes finished after 87 time steps
[ 0.64591106 -0.32893107 -6.9235203  -5.15857704]
Episodes finished after 88 time steps
[ 0.63933243 -0.14719613 -7.02669184 -5.55283017]
Episodes finished after 89 time steps
[ 0.63638851 -0.3487764  -7.13774844 -5.52925616]
Episodes finished after 90 time steps
[ 0.62941298 -0.17623324 -7.24833357 -5.92094003]
Episodes finished after 91 time steps
[ 0.62588832 -0.3825578  -7.3667

array([0.00830886, 0.02003006, 0.00122797, 0.03725499])

In [6]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


In [7]:
print(env.observation_space.high)
print(env.observation_space.low)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


# 2. Defining parameter

In [11]:
#Training Parameters

n_epsd = 1000
n_win_ticks = 195
max_env_steps = None
gamma = 1.0
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01 
alpha_decay = 0.01
batch_size = 64
monitor = False
quiet = False

#Environment Parameters

memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None:
    env.max_episode_steps = max_env_steps


# Building  Neural Network

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


In [15]:
#Model Definition

model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))


# Defining necessary functions

In [16]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(
        memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

In [ ]:
import random
import gym
import math
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Training Parameters
n_episodes=1000
n_win_ticks=195
max_env_steps=None
gamma=1.0
epsilon=1.0
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01
alpha_decay=0.01
batch_size=64
monitor=False
quiet=False

# Environment Parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps

from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Model Definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))

def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(
        memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay    
        
def run():
    scores = deque(maxlen=100)

    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1

        scores.append(i)
        mean_score = np.mean(scores)
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. Solved after {} trials'.format(e, e - 100))
            return e - 100
        if e % 20 == 0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
   
        replay(batch_size, get_epsilon(e))

    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

run()

[Episode 0] - Mean survival time over last 100 episodes was 12.0 ticks.
[Episode 20] - Mean survival time over last 100 episodes was 9.666666666666666 ticks.
[Episode 40] - Mean survival time over last 100 episodes was 9.487804878048781 ticks.
[Episode 60] - Mean survival time over last 100 episodes was 9.40983606557377 ticks.
[Episode 80] - Mean survival time over last 100 episodes was 9.382716049382717 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 9.36 ticks.
[Episode 120] - Mean survival time over last 100 episodes was 9.5 ticks.
[Episode 140] - Mean survival time over last 100 episodes was 9.87 ticks.
[Episode 160] - Mean survival time over last 100 episodes was 9.98 ticks.
[Episode 180] - Mean survival time over last 100 episodes was 10.14 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 10.36 ticks.
[Episode 220] - Mean survival time over last 100 episodes was 10.44 ticks.
[Episode 240] - Mean survival time over last 100 episodes was 10.